## Notebook to demonstrate Data-Services workflow
### The workflow in a nutshell
TAO Data Services include 4 key pipelines:
1. Offline data augmentation using DALI
2. Auto labeling using TAO Mask Auto-labeler (MAL)
3. Annotation conversion
4. Groundtruth analytics

## Learning Objectives

In this notebook, you will learn how to leverage the simplicity and convenience of TAO to:

* Convert KITTI dataset to COCO format
* Run auto-labeling to generate pseudo masks for KITTI bounding boxes
* Apply data augmentation to the KITTI dataset with bounding boxe refinement
* Run data analytics to collect useful statistics on the original and augmented KITTI dataset

### Table of contents

1. [Create a cloud workspace](#head-2)
1. [Convert KITTI data to COCO format](#head-1)
1. [Generate pseudo-masks with the auto-labeler](#head-2)
1. [Apply data augmentation](#head-3)
1. [Perform data analytics](#head-4)
1. [Perform data validation](#head-5)

### Requirements
Please find the server requirements [here](https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_api/api_setup.html#)

In [ ]:
import json
import os
import time
from IPython.display import clear_output

# Import TAO SDK
from tao_sdk.client import TaoClient

In [ ]:
# Restore variable in case of jupyter session restart and resume execution where it left off
%store -r base_url
%store -r headers
%store -r workspace_id
%store -r kitti_dataset_id
%store -r coco_dataset_id
%store -r coco_mask_dataset_id
%store -r coco_mask_augmented_dataset_id
%store -r job_map

### FIXME's <a class="anchor" id="head-1"></a>

1. Assign the ip_address and port_number in FIXME 1 ([info](https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_api/api_rest_api.html))
1. Assign the ngc_key variable in FIXME 2
1. Assign the ngc_org_name variable in FIXME 3
1. Set cloud storage details in FIXME 4
1. Assign path of kitti dataset relative to the bucket in FIXME 5

#### Set API service's host information

In [ ]:
# FIXME 4: Set TAO API environment variables

# Set to your TAO API endpoint
os.environ["TAO_BASE_URL"] = os.environ.get("TAO_BASE_URL", "https://your_tao_ip_address:port/api/v2")

#### Set NGC Personal key for authentication and NGC org to access API services

In [ ]:
# FIXME 5: Your NGC personal key
os.environ["NGC_KEY"] = ngc_key = os.environ.get("NGC_KEY", "your_ngc_key")

In [ ]:
# FIXME 6: Your NGC ORG name
os.environ["NGC_ORG"] = ngc_org_name = os.environ.get("NGC_ORG", "nvstaging")

### Login <a class="anchor" id="head-2"></a>

In [ ]:
# Initialize TAO Client and login using SDK
tao_client = TaoClient()

# Login using TAO SDK - this will automatically save credentials to environment variables
login_response = tao_client.login(
    ngc_key=ngc_key,
    ngc_org_name=ngc_org_name,
    enable_telemetry=True
)

print("Login successful!")
print("JWT Token:", tao_client.token)
print("API Base URL:", tao_client.base_url)
print("Organization:", tao_client.org_name)

%store tao_client

### Get NVCF gpu details <a class="anchor" id="head-2"></a>

 One of the keys of the response json are to be used as platform_id when you run each job

In [ ]:
# # Valid only for NVCF backend during TAO-API helm deployment currently
# # Get available GPU types using TAO SDK
# try:
#     gpu_types = tao_client.get_gpu_types()
#     print("Available GPU types:")
#     print(json.dumps(gpu_types, indent=4))
# except Exception as e:
#     print("Could not fetch GPU types (may not be available on this deployment):", str(e))

### Create cloud workspace
This workspace will be the place where your datasets reside and your results of TAO API jobs will be pushed to.

If you want to have different workspaces for dataset and experiment, duplocate the workspace creation part and adjust the metadata accordingly.

In [ ]:
# FIXME 7: Dataset Cloud bucket details to download dataset or push job artifacts for jobs

cloud_metadata = {}

# A Representative name for this cloud info
os.environ["TAO_WORKSPACE_NAME"] = cloud_metadata["name"] = os.environ.get("TAO_WORKSPACE_NAME", "AWS workspace info")

# Cloud specific details. Below is assuming AWS.
cloud_metadata["cloud_specific_details"] = {}

 # Whether it is AWS, HuggingFace or Azure
os.environ["TAO_WORKSPACE_CLOUD_TYPE"] = cloud_metadata["cloud_specific_details"]["cloud_type"] = os.environ.get("TAO_WORKSPACE_CLOUD_TYPE", "aws")

# Bucket region
os.environ["TAO_WORKSPACE_CLOUD_REGION"] = cloud_metadata["cloud_specific_details"]["cloud_region"] = os.environ.get("TAO_WORKSPACE_CLOUD_REGION", "us-west-1")

# Bucket name
os.environ["TAO_WORKSPACE_CLOUD_BUCKET_NAME"] = cloud_metadata["cloud_specific_details"]["cloud_bucket_name"] = os.environ.get("TAO_WORKSPACE_CLOUD_BUCKET_NAME", "bucket_name")

# Access and Secret keys
os.environ["TAO_WORKSPACE_CLOUD_ACCESS_KEY"] = cloud_metadata["cloud_specific_details"]["access_key"] = os.environ.get("TAO_WORKSPACE_CLOUD_ACCESS_KEY", "access_key")
os.environ["TAO_WORKSPACE_CLOUD_SECRET_KEY"] = cloud_metadata["cloud_specific_details"]["secret_key"] = os.environ.get("TAO_WORKSPACE_CLOUD_SECRET_KEY", "secret_key")

In [ ]:
# Create cloud workspace using TAO SDK
workspace_id = tao_client.create_workspace(
    name=cloud_metadata["name"],
    cloud_type=cloud_metadata["cloud_specific_details"]["cloud_type"],
    cloud_specific_details=cloud_metadata["cloud_specific_details"]
)

print("Workspace created successfully!")
print(f"Workspace ID: {workspace_id}")

# Get workspace details to confirm creation
workspace_details = tao_client.get_workspace_metadata(workspace_id)
print("Workspace details:")
print(json.dumps(workspace_details, indent=4))

%store workspace_id

## 1. Convert KITTI data to COCO format <a class="anchor" id="head-1"></a>
We would first convert the dataset from KITTI to COCO formats.

### Create the dataset
We support both KITTI and COCO data formats

KITTI dataset follow the directory structure displayed below:
```
$DATA_DIR/dataset
 images
   image_name_1.jpg
   image_name_2.jpg
| ...
 labels
 image_name_1.txt
 image_name_2.txt
 ...
```

And COCO dataset follow the directory structure displayed below:
```
$DATA_DIR/dataset
 images
   image_name_1.jpg
   image_name_2.jpg
| ...
 annotations.json
```
For this notebook, we will be using the kitti object detection dataset for this example. To find more details, please visit [here](http://www.cvlibs.net/datasets/kitti/eval_object.php?obj_benchmark=2d).

In [ ]:
job_map = {}

In [ ]:
# FIXME5 : Set path relative to cloud bucket
os.environ["TAO_KITTI_DATASET_PATH"] = kitti_dataset_path = os.environ.get("TAO_KITTI_DATASET_PATH", "/data/tao_od_synthetic_subset_train_convert_cleaned/")

In [ ]:
# Create train dataset using TAO SDK
kitti_dataset_id = tao_client.create_dataset(
    dataset_type="object_detection",
    dataset_format="kitti",
    workspace_id=workspace_id,
    cloud_file_path=kitti_dataset_path,
    use_for=["testing"]
)

print("Kitti dataset created successfully!")
print(f"Kitti Dataset ID: {kitti_dataset_id}")

%store kitti_dataset_id

In [ ]:
# Check kitti dataset progress using TAO SDK
while True:
    clear_output(wait=True)
    dataset_details = tao_client.get_dataset_metadata(kitti_dataset_id)
    
    print(f"Kitti Dataset Status: {dataset_details.get('status', 'Unknown')}")
    print(f"Dataset ID: {kitti_dataset_id}")
    
    if dataset_details.get("status") == "invalid_pull":
        print("Dataset pull failed!")
        validation_details = dataset_details.get("validation_details", {})
        if validation_details:
            print("Validation details:")
            print(json.dumps(validation_details, indent=4))
        raise ValueError("Dataset pull failed")
        
    if dataset_details.get("status") == "pull_complete":
        print("Kitti dataset pull completed successfully!")
        print("Dataset details:")
        print(json.dumps(dataset_details, indent=4))
        break
        
    time.sleep(5)

### Dataset format conversion action 


#### Get specs


In [ ]:
# Get default anootations conversion specs using TAO SDK
annotations_conversion_spec_response = tao_client.get_job_schema(action="annotation_format_convert", network_arch="object_detection")
annotations_conversion_specs = annotations_conversion_spec_response.get("default", {})
print("Default annotations conversion specifications:")
print(json.dumps(annotations_conversion_specs, sort_keys=True, indent=4))

In [ ]:
# Updating spec file
annotations_conversion_specs["data"]["input_format"] = "KITTI"
annotations_conversion_specs["data"]["output_format"] = "COCO"
print(json.dumps(annotations_conversion_specs, sort_keys=True, indent=4))

#### Run action 


In [ ]:
action = "annotation_format_convert"
train_ds_convert_job_name = f"annotations_convert_job"

train_ds_convert_job_id = tao_client.create_job(
    kind="dataset",
    name=train_ds_convert_job_name,
    network_arch="object_detection",
    workspace=workspace_id,
    dataset_id=kitti_dataset_id,
    action=action,
    specs=annotations_conversion_specs,  # Pass as dict, not JSON string
    # platform_id="9af1aa90-8ea5-5a11-98d9-3879cd0da92c",  # Optional: Pick from get_gpu_types output
)

print("Train Dataset Convert job created successfully!")
print(f"Train Dataset Convert Job ID: {train_ds_convert_job_id}")
print(f"Action: train dataset_convert")

job_map["annotations_convert_job"] = train_ds_convert_job_id
print("\nUpdated Job Map:")
print(json.dumps(job_map, indent=4))
%store job_map

In [ ]:
# Monitor job status by repeatedly running this cell
coco_dataset_id = kitti_dataset_id
%store coco_dataset_id
annotations_convert_job_id = job_map["annotations_convert_job"]

while True:    
    clear_output(wait=True)
    
    try:
        job_status = tao_client.get_job_metadata(annotations_convert_job_id)
        
        print(f"Annotations Convert Job Status")
        print(f"Job ID: {annotations_convert_job_id}")
        print(f"Status: {job_status.get('status', 'Unknown')}")
        print(f"Progress: {job_status.get('progress', 'N/A')}")
        
        # Show detailed status information
        print("\nDetailed Status:")
        print(json.dumps(job_status.get("job_details", {}), sort_keys=True, indent=4))
        
        current_status = job_status.get("status", "Unknown")
        
        if current_status == "Error":
            raise Exception("Annotations Convert job failed!")
            
        if current_status in ["Done", "Completed"]:
            print("Annotations Convert job completed successfully!")
            break
            
        if current_status in ["Canceled", "Paused"]:
            print(f"Annotations Convert job {current_status}")
            break
            
    except Exception as e:
        if "failed" in str(e).lower():
            raise
        print(f" Error fetching inference job status: {str(e)}")
        print("Job might still be starting up...")
        
    time.sleep(15)

In [ ]:
# After the action is completed the format of dataset will be converted to coco from kitti
coco_dataset_details = tao_client.get_dataset_metadata(coco_dataset_id)
print(json.dumps(coco_dataset_details, indent=4))

## 2. Generate pseudo-masks with the auto-labeler <a class="anchor" id="head-2"></a>
Here we will use a pretrained MAL model to generate pseudo-masks for the converted KITTI data. 

### Create a coco Dataset - If you already have data in coco detection format(without masks) and skipped step 1

In [ ]:
# # Create train dataset using TAO SDK
# coco_dataset_id = tao_client.create_dataset(
#     dataset_type="object_detection",
#     dataset_format="coco",
#     workspace_id=workspace_id,
#     cloud_file_path=coco_dataset_path,
#     use_for=["testing"]
# )

# print("Coco dataset created successfully!")
# print(f"Coco Dataset ID: {coco_dataset_id}")

# %store coco_dataset_id

In [ ]:
# # Check kitti dataset progress using TAO SDK
# while True:
#     clear_output(wait=True)
#     dataset_details = tao_client.get_dataset_metadata(coco_dataset_id)
    
#     print(f"Coco Dataset Status: {dataset_details.get('status', 'Unknown')}")
#     print(f"Dataset ID: {coco_dataset_id}")
    
#     if dataset_details.get("status") == "invalid_pull":
#         print("Dataset pull failed!")
#         validation_details = dataset_details.get("validation_details", {})
#         if validation_details:
#             print("Validation details:")
#             print(json.dumps(validation_details, indent=4))
#         raise ValueError("Dataset pull failed")
        
#     if dataset_details.get("status") == "pull_complete":
#         print("Coco dataset pull completed successfully!")
#         print("Dataset details:")
#         print(json.dumps(dataset_details, indent=4))
#         break
        
#     time.sleep(5)

### Assign PTM

In [ ]:
# List base experiments (PTMs) using TAO SDK  
# These are the pre-trained models available for the selected network architecture
base_experiments = tao_client.list_base_experiments(filter_params={"network_arch": "auto_label"})

print(f" Available base experiments (PTMs) for auto_label:")
print("name\t\t\t     model id\t\t\t     network architecture")
print("-" * 120)

for exp in base_experiments:
    exp_name = exp.get("name", "N/A")
    exp_id = exp.get("id", "N/A")
    exp_arch = exp.get("network_arch", "N/A")
    print(f"{exp_name}\t{exp_id}\t{exp_arch}")

In [ ]:
pretrained_map = {"auto_label" : "mask_auto_label:trainable_v1.1"}

In [ ]:
# Get pretrained model using TAO SDK
selected_ptm_id = None
base_experiments_detailed = tao_client.list_base_experiments(filter_params={"network_arch": "mal"})

# Search for PTM with given NGC path
for exp in base_experiments_detailed:
    ngc_path = exp.get("ngc_path", "")
    if ngc_path.endswith(pretrained_map["auto_label"]):
        selected_ptm_id = exp.get("id")
        print("Selected PTM metadata:")
        print(json.dumps(exp, indent=4))
        break

if not selected_ptm_id:
    print(f" PTM with NGC path ending in '{pretrained_map["auto_label"]}' not found!")

In [ ]:
print(f"Selected PTM ID: {selected_ptm_id}")

if selected_ptm_id:
    update_payload = {"base_experiment_ids": [selected_ptm_id]}
    
    # Assign PTM to dataset using TAO SDK
    updated_dataset = tao_client.update_dataset_metadata(
        coco_dataset_id, 
        update_payload
    )
    print("\nUpdated dataset:")
    print(json.dumps(updated_dataset, indent=4))
else:
    print("ERROR: selected_ptm_id is None! The PTM was not found.")
    print("Make sure you ran the cell that searches for the PTM first.")

### Auto labeling action

#### Get specs

In [ ]:
# Get default anootations conversion specs using TAO SDK
auto_label_spec_response = tao_client.get_job_schema(action="auto_label", network_arch="object_detection")
auto_label_generate_specs = auto_label_spec_response.get("default", {})
print("Default annotations conversion specifications:")
print(json.dumps(auto_label_generate_specs, sort_keys=True, indent=4))

In [ ]:
# Override any of the parameters listed in the previous cell as required
auto_label_generate_specs["gpu_ids"] = [0]
print(json.dumps(auto_label_generate_specs, sort_keys=True, indent=4))

#### Run action

In [ ]:
# Run action
parent = job_map["annotations_convert_job"]
action = "auto_label"
auto_label_job_name = f"auto_label_job"

auto_label_job_id = tao_client.create_job(
    kind="dataset",
    name=auto_label_job_name,
    network_arch="object_detection",
    workspace=workspace_id,
    dataset_id=coco_dataset_id,
    action=action,
    parent_job_id=parent,
    specs=auto_label_generate_specs,  # Pass as dict, not JSON string
    # platform_id="9af1aa90-8ea5-5a11-98d9-3879cd0da92c",  # Optional: Pick from get_gpu_types output
)

print("Auto Labeling job created successfully!")
print(f"Auto Labeling Job ID: {auto_label_job_id}")
print(f"Action: auto_label")

job_map["auto_labeling"] = auto_label_job_id
print("\nUpdated Job Map:")
print(json.dumps(job_map, indent=4))
%store job_map

In [ ]:
# Monitor job status by repeatedly running this cell
coco_mask_dataset_id = kitti_dataset_id
%store coco_mask_dataset_id
auto_labeling_job_id = job_map["auto_labeling"]

while True:    
    clear_output(wait=True)
    
    try:
        job_status = tao_client.get_job_metadata(auto_labeling_job_id)
        
        print(f"Auto Labeling Job Status")
        print(f"Job ID: {auto_labeling_job_id}")
        print(f"Status: {job_status.get('status', 'Unknown')}")
        print(f"Progress: {job_status.get('progress', 'N/A')}")
        
        # Show detailed status information
        print("\nDetailed Status:")
        print(json.dumps(job_status.get("job_details", {}), sort_keys=True, indent=4))
        
        current_status = job_status.get("status", "Unknown")
        
        if current_status == "Error":
            raise Exception("Auto Labeling job failed!")
            
        if current_status in ["Done", "Completed"]:
            print("Auto Labeling job completed successfully!")
            break
            
        if current_status in ["Canceled", "Paused"]:
            print(f"Auto Labeling job {current_status}")
            break
            
    except Exception as e:
        if "failed" in str(e).lower():
            raise
        print(f" Error fetching inference job status: {str(e)}")
        print("Job might still be starting up...")
        
    time.sleep(15)

## 3. Apply data augmentation <a class="anchor" id="head-3"></a>
In this section, we run offline augmentation with the original dataset. During the augmentation process, we can use the pseudo-masks generated from the last step to refine the distorted or rotated bounding boxes

### Create a coco mask Dataset - If you already have data in coco segmentation format and skipped step 1 and 2

In [ ]:
# # Create train dataset using TAO SDK
# coco_mask_dataset_id = tao_client.create_dataset(
#     dataset_type="object_detection",
#     dataset_format="coco",
#     workspace_id=workspace_id,
#     cloud_file_path=coco_mask_dataset_path,
#     use_for=["testing"]
# )

# print("Coco Mask dataset created successfully!")
# print(f"Coco Mask Dataset ID: {coco_mask_dataset_id}")

# %store coco_mask_dataset_id

In [ ]:
# # Check coco mask dataset progress using TAO SDK
# while True:
#     clear_output(wait=True)
#     dataset_details = tao_client.get_dataset_metadata(coco_mask_dataset_id)
    
#     print(f"Coco Mask Dataset Status: {dataset_details.get('status', 'Unknown')}")
#     print(f"Dataset ID: {coco_mask_dataset_id}")
    
#     if dataset_details.get("status") == "invalid_pull":
#         print("Dataset pull failed!")
#         validation_details = dataset_details.get("validation_details", {})
#         if validation_details:
#             print("Validation details:")
#             print(json.dumps(validation_details, indent=4))
#         raise ValueError("Dataset pull failed")
        
#     if dataset_details.get("status") == "pull_complete":
#         print("Coco Mask dataset pull completed successfully!")
#         print("Dataset details:")
#         print(json.dumps(dataset_details, indent=4))
#         break
        
#     time.sleep(5)

### Run data augmentation action


#### Get specs


In [ ]:
# Get default anootations conversion specs using TAO SDK
augmentation_spec_response = tao_client.get_job_schema(action="augment", network_arch="object_detection")
augmentation_generate_specs = augmentation_spec_response.get("default", {})
print("Default data augmentation specifications:")
print(json.dumps(augmentation_generate_specs, sort_keys=True, indent=4))

In [ ]:
# Make changes to the specs if necessary
print(json.dumps(augmentation_generate_specs, sort_keys=True, indent=4))

#### Run action


In [ ]:
# Run action
parent = job_map["auto_labeling"]
action = "augment"
augment_job_name = f"augment_job"

augment_job = tao_client.create_job(
    kind="dataset",
    name=augment_job_name,
    network_arch="object_detection",
    workspace=workspace_id,
    dataset_id=coco_mask_dataset_id,
    action=action,
    parent_job_id=parent,
    specs=auto_label_generate_specs,  # Pass as dict, not JSON string
    # platform_id="9af1aa90-8ea5-5a11-98d9-3879cd0da92c",  # Optional: Pick from get_gpu_types output
)

print("Augmentation job created successfully!")
print(f"Augmentation Job ID: {augment_job}")
print(f"Action: augment")

job_map["augment"] = augment_job
print("\nUpdated Job Map:")
print(json.dumps(job_map, indent=4))
%store job_map

In [ ]:
# Monitor job status by repeatedly running this cell
augment_job_id = job_map["augment"]
coco_mask_augmented_dataset_id = augment_job_id
%store coco_mask_augmented_dataset_id

while True:    
    clear_output(wait=True)
    
    try:
        job_status = tao_client.get_job_metadata(augment_job_id)
        
        print(f"Augmentation Job Status")
        print(f"Job ID: {augment_job_id}")
        print(f"Status: {job_status.get('status', 'Unknown')}")
        print(f"Progress: {job_status.get('progress', 'N/A')}")
        
        # Show detailed status information
        print("\nDetailed Status:")
        print(json.dumps(job_status.get("job_details", {}), sort_keys=True, indent=4))
        
        current_status = job_status.get("status", "Unknown")
        
        if current_status == "Error":
            raise Exception("Augmentation job failed!")
            
        if current_status in ["Done", "Completed"]:
            print("Augmentation job completed successfully!")
            break
            
        if current_status in ["Canceled", "Paused"]:
            print(f"Augmentation job {current_status}")
            break
            
    except Exception as e:
        if "failed" in str(e).lower():
            raise
        print(f" Error fetching inference job status: {str(e)}")
        print("Job might still be starting up...")
        
    time.sleep(15)

## 4. Perform data analytics  <a class="anchor" id="head-4"></a>
Next, we perform analytics with the KITTI dataset.

### Run Data analytics annotation analytics action


#### Get specs


In [ ]:
# Get default anootations conversion specs using TAO SDK
analytics_spec_response = tao_client.get_job_schema(action="analyze", network_arch="object_detection")
analytics_analyze_specs = analytics_spec_response.get("default", {})
print("Default data analytics specifications:")
print(json.dumps(analytics_analyze_specs, sort_keys=True, indent=4))

In [ ]:
# Make changes to the specs if necessary
analytics_analyze_specs["data"]["input_format"] = "COCO"
print(json.dumps(analytics_analyze_specs, sort_keys=True, indent=4))

#### Run action


In [ ]:
# Run action
parent = job_map["annotations_convert_job"]
action = "analyze"
analytics_job_name = f"analytics_job"

analytics_job = tao_client.create_job(
    kind="dataset",
    name=analytics_job_name,
    network_arch="object_detection",
    workspace=workspace_id,
    dataset_id=kitti_dataset_id,
    action=action,
    parent_job_id=parent,
    specs=auto_label_generate_specs,  # Pass as dict, not JSON string
    # platform_id="9af1aa90-8ea5-5a11-98d9-3879cd0da92c",  # Optional: Pick from get_gpu_types output
)

print("Analytics job created successfully!")
print(f"Analytics Job ID: {analytics_job}")
print(f"Action: analyze")

job_map["analytics"] = analytics_job
print("\nUpdated Job Map:")
print(json.dumps(job_map, indent=4))
%store job_map

In [ ]:
# Monitor job status by repeatedly running this cell
analytics_job_id = job_map["analytics"]

while True:    
    clear_output(wait=True)
    
    try:
        job_status = tao_client.get_job_metadata(analytics_job_id)
        
        print(f"Analytics Job Status")
        print(f"Job ID: {analytics_job_id}")
        print(f"Status: {job_status.get('status', 'Unknown')}")
        print(f"Progress: {job_status.get('progress', 'N/A')}")
        
        # Show detailed status information
        print("\nDetailed Status:")
        print(json.dumps(job_status.get("job_details", {}), sort_keys=True, indent=4))
        
        current_status = job_status.get("status", "Unknown")
        
        if current_status == "Error":
            raise Exception("Analytics job failed!")
            
        if current_status in ["Done", "Completed"]:
            print("Analytics job completed successfully!")
            break
            
        if current_status in ["Canceled", "Paused"]:
            print(f"Analytics job {current_status}")
            break
            
    except Exception as e:
        if "failed" in str(e).lower():
            raise
        print(f" Error fetching inference job status: {str(e)}")
        print("Job might still be starting up...")
        
    time.sleep(15)

## 5. Perform data validation  <a class="anchor" id="head-5"></a>
Next, we perform validate the annotations and images.

### Run Data annotation validation action


#### Get specs


In [ ]:
# Get default anootations conversion specs using TAO SDK
validate_annotation_spec_response = tao_client.get_job_schema(action="validate_annotations", network_arch="object_detection")
validate_annotation_specs = validate_annotation_spec_response.get("default", {})
print("Default validate annotation specifications:")
print(json.dumps(validate_annotation_specs, sort_keys=True, indent=4))

In [ ]:
# Make changes to the specs if necessary
validate_annotation_specs["data"]["input_format"] = "COCO"
print(json.dumps(validate_annotation_specs, sort_keys=True, indent=4))

#### Run action


In [ ]:
# Run action
parent = job_map["annotations_convert_job"]
action = "validate_annotations"
data_validate_job_name = f"validate_job"

validate_job = tao_client.create_job(
    kind="dataset",
    name=data_validate_job_name,
    network_arch="object_detection",
    workspace=workspace_id,
    dataset_id=kitti_dataset_id,
    action=action,
    parent_job_id=parent,
    specs=auto_label_generate_specs,  # Pass as dict, not JSON string
    # platform_id="9af1aa90-8ea5-5a11-98d9-3879cd0da92c",  # Optional: Pick from get_gpu_types output
)

print("Data validation job created successfully!")
print(f"Data validation Job ID: {validate_job}")
print(f"Action: validate")

job_map["data_validation"] = validate_job
print("\nUpdated Job Map:")
print(json.dumps(job_map, indent=4))
%store job_map

In [ ]:
# Monitor job status by repeatedly running this cell
data_validate_job_id = job_map["data_validation"]

while True:    
    clear_output(wait=True)
    
    try:
        job_status = tao_client.get_job_metadata(data_validate_job_id)
        
        print(f"Data validation Job Status")
        print(f"Job ID: {data_validate_job_id}")
        print(f"Status: {job_status.get('status', 'Unknown')}")
        print(f"Progress: {job_status.get('progress', 'N/A')}")
        
        # Show detailed status information
        print("\nDetailed Status:")
        print(json.dumps(job_status.get("job_details", {}), sort_keys=True, indent=4))
        
        current_status = job_status.get("status", "Unknown")
        
        if current_status == "Error":
            raise Exception("Data validation job failed!")
            
        if current_status in ["Done", "Completed"]:
            print("Data validation job completed successfully!")
            break
            
        if current_status in ["Canceled", "Paused"]:
            print(f"Data validation job {current_status}")
            break
            
    except Exception as e:
        if "failed" in str(e).lower():
            raise
        print(f" Error fetching inference job status: {str(e)}")
        print("Job might still be starting up...")
        
    time.sleep(15)

### Run Data image validation action - removes corrupted images and creates a new dataset

#### Get specs


In [ ]:
# Get default anootations conversion specs using TAO SDK
validate_images_spec_response = tao_client.get_job_schema(action="validate_images", network_arch="object_detection")
validate_images_specs = validate_images_spec_response.get("default", {})
print("Default validate images specifications:")
print(json.dumps(validate_images_specs, sort_keys=True, indent=4))

In [ ]:
# Make changes to the specs if necessary

#### Run action


In [ ]:
# Run action
action = "validate_images"
validate_images_job_name = f"validate_images_job"

validate_images_job = tao_client.create_job(
    kind="dataset",
    name=validate_images_job_name,
    network_arch="object_detection",
    workspace=workspace_id,
    dataset_id=kitti_dataset_id,
    action=action,
    specs=validate_images_specs,  # Pass as dict, not JSON string
    # platform_id="9af1aa90-8ea5-5a11-98d9-3879cd0da92c",  # Optional: Pick from get_gpu_types output
)

print("Validate images job created successfully!")
print(f"Validate images Job ID: {validate_images_job}")
print(f"Action: validate_images")

job_map["validate_images"] = validate_images_job
print("\nUpdated Job Map:")
print(json.dumps(job_map, indent=4))
%store job_map

In [ ]:
# Monitor job status by repeatedly running this cell
validate_images_job_id = job_map["validate_images"]

while True:    
    clear_output(wait=True)
    
    try:
        job_status = tao_client.get_job_metadata(validate_images_job_id)
        
        print(f"Validate images Job Status")
        print(f"Job ID: {validate_images_job_id}")
        print(f"Status: {job_status.get('status', 'Unknown')}")
        print(f"Progress: {job_status.get('progress', 'N/A')}")
        
        # Show detailed status information
        print("\nDetailed Status:")
        print(json.dumps(job_status.get("job_details", {}), sort_keys=True, indent=4))
        
        current_status = job_status.get("status", "Unknown")
        
        if current_status == "Error":
            raise Exception("Validate images job failed!")
            
        if current_status in ["Done", "Completed"]:
            print("Validate images job completed successfully!")
            break
            
        if current_status in ["Canceled", "Paused"]:
            print(f"Validate images job {current_status}")
            break
            
    except Exception as e:
        if "failed" in str(e).lower():
            raise
        print(f" Error fetching inference job status: {str(e)}")
        print("Job might still be starting up...")
        
    time.sleep(15)

### Delete Jobs<a class="anchor" id="head-22"></a>

In [ ]:
# Delete jobs instead of experiments
# Delete all created jobs using TAO SDK

print(" Deleting all created jobs...")

jobs_to_delete = []
for job_key, job_id in job_map.items():
    try:
        delete_result = tao_client.delete_job(job_id)
        print(f" Deleted job: {job_key} (ID: {job_id})")
    except Exception as e:
        print(f" Failed to delete job {job_key} (ID: {job_id}): {str(e)}")

print(f"\n Job cleanup completed! Processed {len(jobs_to_delete)} jobs.")

### Delete dataset <a class="anchor" id="head-21"></a>

#### Delete original kitti dataset <a class="anchor" id="head-21"></a>

In [ ]:
# Delete kitti dataset using TAO SDK
try:
    delete_result = tao_client.delete_dataset(kitti_dataset_id)
    print("Kitti dataset deleted successfully!")
    print(f"Dataset ID: {kitti_dataset_id}")
    if delete_result:
        print("Delete Response:")
        print(json.dumps(delete_result, indent=4))
except Exception as e:
    print(f" Failed to delete kitti dataset {kitti_dataset_id}: {str(e)}")

#### Delete coco augment dataset <a class="anchor" id="head-21"></a>

In [ ]:
# Delete coco augmented dataset using TAO SDK
try:
    delete_result = tao_client.delete_dataset(coco_mask_augmented_dataset_id)
    print("Coco mask dataset deleted successfully!")
    print(f"Dataset ID: {coco_mask_augmented_dataset_id}")
    if delete_result:
        print("Delete Response:")
        print(json.dumps(delete_result, indent=4))
except Exception as e:
    print(f" Failed to delete train dataset {coco_dataset_id}: {str(e)}")